In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import os

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def loadNpy(filename):
    with open(filename, "rb") as f: return np.load(f)

db = f"{os.getcwd()}/../data/slices/gaussian-grabber/size160000-hw40-d100/"
data_type = ""

In [4]:
X_train, X_val, X_test = loadNpy(db + f"X_train.npy"), loadNpy(db + f"X_val.npy"), loadNpy(db + f"X_test.npy")
y_train, y_val, y_test = loadNpy(db + f"c-y{data_type}_train.npy"), loadNpy(db + f"c-y{data_type}_val.npy"), loadNpy(db + f"c-y{data_type}_test.npy")

In [5]:
def npy_to_tensor(data, l=False):
    if not l:
        data = torch.from_numpy(data).float()
    else:
        data = torch.from_numpy(data).type(torch.LongTensor)
    return data

In [6]:
X_train, X_val, X_test = npy_to_tensor(X_train), npy_to_tensor(X_val), npy_to_tensor(X_test)
y_train, y_val, y_test = npy_to_tensor(y_train, 1), npy_to_tensor(y_val, 1), npy_to_tensor(y_test, 1)

In [8]:
batch_size = 64

In [9]:
train_dataset = TensorDataset(X_train, y_train)

val_dataset = TensorDataset(X_val, y_val)

test_dataset = TensorDataset(X_test, y_test)

# Data loader
train_iterator = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

val_iterator = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

test_iterator = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [10]:
X_train.shape

torch.Size([1200, 160000])

In [10]:
np.mean(y_train)

TypeError: mean() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, dtype=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [107]:
np.mean(y_val)

0.18666666666666668

In [108]:
np.mean(y_test)

0.15666666666666668

In [20]:
input_size = 160000
h1_size = 20000
h2_size = 100
h3_size = 100
h4_size = 100
h5_size = 100
num_classes = 2
num_epochs = 1000
batch_size = 64
learning_rate = 0.001

In [21]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size).to(device)
        #self.bn1 = nn.BatchNorm1d(h1_size)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(h1_size, h2_size).to(device)
        #self.bn2 = nn.BatchNorm1d(h2_size)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(p=0.5)
        
        #self.drop1 = nn.Dropout(p=0.1)
        #self.bn2 = nn.BatchNorm1d(h2_size)
        self.fc3 = nn.Linear(h2_size, h3_size).to(device)
        #self.bn3 = nn.BatchNorm1d(h3_size)
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(p=0.5)
        
        #self.bn3 = nn.BatchNorm1d(h3_size)
        self.fc4 = nn.Linear(h3_size, h4_size).to(device)
        #self.bn4 = nn.BatchNorm1d(h4_size)
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(p=0.5)
        
        #self.bn4 = nn.BatchNorm1d(h4_size)
        #self.drop2 = nn.Dropout(p=0.1)
        self.fc5 = nn.Linear(h4_size, h5_size).to(device)
        #self.bn5 = nn.BatchNorm1d(h4_size)
        self.relu5 = nn.ReLU()
        self.drop5 = nn.Dropout(p=0.5)
        
        self.fc6 = nn.Linear(h2_size, num_classes).to(device)
    
    def forward(self, x):
        out = self.fc1(x)
        #out = self.bn1(out)
        out = self.relu1(out)
        out = self.drop1(out)
        
        out = self.fc2(out)
        #out = self.bn2(out)
        out = self.relu2(out)
        out = self.drop2(out)
        
        #out = self.bn2(out)
        out = self.fc3(out)
        #out = self.bn3(out)
        out = self.relu3(out)
        out = self.drop3(out)
        #out = self.bn3(out)
        
        out = self.fc4(out)
        #out = self.bn4(out)
        out = self.relu4(out)
        out = self.drop4(out)
        
        #out = self.bn4(out)
        out = self.fc5(out)
        #out = self.bn5(out)
        out = self.relu5(out)
        out = self.drop5(out)
        
        out = self.fc6(out)
        return out

In [22]:
def eval_nn(mod, data):
    preds = []
    actual = []
    
    shape = data.dataset.tensors[0].shape[1:]
    
    mod.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in data:
            images = images.to(device)
            labels = labels.to(device)
            outputs = mod(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            preds.append(predicted)
            actual.append(labels)

        print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')
    return preds, actual, correct/total

In [23]:
nn_model = NeuralNet(input_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate, weight_decay=1e-3)

In [24]:
nn_epochs = {}

# Train the model
total_step = len(train_iterator)
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_iterator):  
        # Move tensors to the configured device
        #print(images.shape)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        #print(images.shape)
        outputs = nn_model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(i)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    nn_epochs[epoch] = {"loss": loss.item(), "train": eval_nn(nn_model, train_iterator), 
                 "val": eval_nn(nn_model, val_iterator), "test": eval_nn(nn_model, test_iterator)}
    nn_model.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000], Step [19/19], Loss: 99.9551
Accuracy of the network on the 1200 test images: 84.66666666666667 %
Accuracy of the network on the 400 test images: 86.75 %
Accuracy of the network on the 400 test images: 84.25 %
Epoch [2/1000], Step [19/19], Loss: 4.5253
Accuracy of the network on the 1200 test images: 84.58333333333333 %
Accuracy of the network on the 400 test images: 86.75 %
Accuracy of the network on the 400 test images: 84.25 %
Epoch [3/1000], Step [19/19], Loss: 0.6380
Accuracy of the network on the 1200 test images: 84.58333333333333 %
Accuracy of the network on the 400 test images: 86.75 %
Accuracy of the network on the 400 test images: 84.25 %
Epoch [4/1000], Step [19/19], Loss: 0.6158
Accuracy of the network on the 1200 test images: 84.58333333333333 %
Accuracy of the network on the 400 test images: 86.75 %
Accuracy of the network on the 400 test images: 84.25 %
Epoch [5/1000], Step [19/19], Loss: 1.2726
Accuracy of the network on the 1200 test images: 84.58333333

KeyboardInterrupt: 

In [ ]:
eval_nn(nn_model, val_iterator)

In [9]:
loaders = {
    'train': train_iterator,
    'val': val_iterator,
    'test': test_iterator
}

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=40,              
                out_channels=64,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),
            #nn.BatchNorm2d(16),
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        ).to(device)
        self.conv2 = nn.Sequential(         
            nn.Conv2d(64, 64, 5, 1, 2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            nn.MaxPool2d(2),                
        ).to(device)
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 256, 5, 1, 2),
            #nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            #nn.MaxPool2d(3),
        ).to(device)
        """
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 128, 5, 1, 2),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            #nn.MaxPool2d(2),
        ).to(device)
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 128, 5, 1, 2),
            nn.ReLU(),
        ).to(device)
        self.conv6 = nn.Sequential(
            nn.Conv2d(128, 512, 5, 1, 2),
            nn.ReLU(),
            #nn.Dropout(p=0.1)
        ).to(device)
        self.conv7 = nn.Sequential(
            nn.Conv2d(512, 256, 5, 1, 2),
            nn.ReLU(),
        ).to(device)
        self.conv8 = nn.Sequential(         
            nn.Conv2d(256, 128, 5, 1, 2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            #nn.MaxPool2d(2),                
        ).to(device)
        self.conv9 = nn.Sequential(         
            nn.Conv2d(128, 512, 5, 1, 2),
            #nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            #nn.MaxPool2d(3),
        ).to(device)
        self.conv10 = nn.Sequential(
            nn.Conv2d(512, 128, 5, 1, 2),
            nn.ReLU(),
            #nn.Dropout(p=0.1),
            #nn.MaxPool2d(2),
        ).to(device)
        self.conv11 = nn.Sequential(
            nn.Conv2d(128, 64, 5, 1, 2),
            nn.ReLU(),
        ).to(device)
        """
        
        # fully connected layer, output 10 classes
        self.out = nn.Linear(64000, 3).to(device)
        
        
    def forward(self, x):
        x = x.to(device)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        """
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        """
        
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        output = self.out(x)
        return output, x    # return x for visualization

In [11]:
from torchmetrics import ROC

def eval_cnn(data):
    
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        pred_ys = []
        labs = []
        for images, labels in loaders[data]:
            images = images.reshape(-1,40,40,100).to(device)
            labels = labels.to(device)
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            pred_ys = pred_ys + test_output.flatten().tolist()
            labs = labs + labels.tolist()
            total += len(labels)
            correct += (pred_y == labels).sum().item()
            pass
        print(f'{data} Accuracy of the model on the {total} {data} images: %.3f' % (correct/total))
        return pred_ys, labs, correct/total
    pass

In [12]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(40, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (out): Linear(in_features=64000, out_features=3, bias=True)
)


In [13]:
loss_func = nn.CrossEntropyLoss()   
loss_func

CrossEntropyLoss()

In [14]:
optimizer = optim.Adam(cnn.parameters(), lr = 0.001)   
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [15]:
from torch.autograd import Variable

num_epochs = 1000
cnn_epochs = {}

def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in tqdm(range(num_epochs)):
        for i, (images, labels) in enumerate(loaders['train']):
            
            images = images.reshape(-1,40,40,100).to(device)
            labels = labels.to(device)
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            #print(i)
            if (i) % 10 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                cnn_epochs[epoch] = {"loss": loss.item(), "train": eval_cnn("train"), 
                             "val": eval_cnn("val"), "test": eval_cnn("test")}
                cnn.train()
                pass
        pass
    pass

train(num_epochs, cnn, loaders)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000], Step [1/12], Loss: 84.3675
train Accuracy of the model on the 1200 train images: 0.396
val Accuracy of the model on the 400 val images: 0.400
test Accuracy of the model on the 400 test images: 0.400
Epoch [1/1000], Step [11/12], Loss: 5.2603
train Accuracy of the model on the 1200 train images: 0.411
val Accuracy of the model on the 400 val images: 0.417
test Accuracy of the model on the 400 test images: 0.398
Epoch [2/1000], Step [1/12], Loss: 2.5850
train Accuracy of the model on the 1200 train images: 0.215
val Accuracy of the model on the 400 val images: 0.195
test Accuracy of the model on the 400 test images: 0.220
Epoch [2/1000], Step [11/12], Loss: 1.7068
train Accuracy of the model on the 1200 train images: 0.385
val Accuracy of the model on the 400 val images: 0.357
test Accuracy of the model on the 400 test images: 0.367
Epoch [3/1000], Step [1/12], Loss: 1.2692
train Accuracy of the model on the 1200 train images: 0.444
val Accuracy of the model on the 400 va

train Accuracy of the model on the 1200 train images: 0.875
val Accuracy of the model on the 400 val images: 0.455
test Accuracy of the model on the 400 test images: 0.415
Epoch [20/1000], Step [11/12], Loss: 0.2552
train Accuracy of the model on the 1200 train images: 0.873
val Accuracy of the model on the 400 val images: 0.455
test Accuracy of the model on the 400 test images: 0.422
Epoch [21/1000], Step [1/12], Loss: 0.3763
train Accuracy of the model on the 1200 train images: 0.778
val Accuracy of the model on the 400 val images: 0.468
test Accuracy of the model on the 400 test images: 0.445
Epoch [21/1000], Step [11/12], Loss: 0.2915
train Accuracy of the model on the 1200 train images: 0.912
val Accuracy of the model on the 400 val images: 0.425
test Accuracy of the model on the 400 test images: 0.448
Epoch [22/1000], Step [1/12], Loss: 0.2192
train Accuracy of the model on the 1200 train images: 0.890
val Accuracy of the model on the 400 val images: 0.430
test Accuracy of the mo

KeyboardInterrupt: 

In [28]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      

NameError: name 'partial' is not defined